In [225]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

In [226]:
link = 'https://www.themoviedb.org/movie/'
links = []
for i in range(10000,10010):
    links.append(link + str(i))
len(links)

10

In [227]:
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'it-IT,it;q=0.9,en-US;q=0.8,en;q=0.7',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'
    }

In [228]:
response = requests.get("https://www.themoviedb.org/movie", headers=headers)
response.status_code
dwn_content = response.text
test_doc = BeautifulSoup(dwn_content, 'html.parser')

In [229]:

def get_page_content(url):
    # In this case , we are going to give request.get function headers to avoid the Status code Error 403

    get_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}
    response_page = requests.get(url, headers = get_headers )
    # we are going to raise exception here if status code gives any value other than 200.
    if not response_page.ok:
        raise Exception ("Failed to request the data. Status Code:- {}".format(response_page.status_code))
    else:
        page_content = response_page.text
        doc_page = BeautifulSoup(page_content, "html.parser")
        return doc_page
popular_shows_url = "https://www.themoviedb.org/movie"
doc = get_page_content(popular_shows_url)
doc.title.text
doc.find_all('div', {'class': 'card style_1'})[0].h2.text
doc.find_all('div', {'class': 'user_score_chart'})[0]['data-percent']

'65'

In [230]:
def empty_dict():
    scraped_dict = {  
                    'Title': [], # ok
                    'Genre': [],  # ok 
                    'Duration': [], # ok
                    'Release_date': [], # ok
                    'User_rating': [], # ok
                    'Tagline': [], # ok
                    'Description': [], # ok
                    'Directors': [], # ok
                    'Novel': [], # ok
                    'Writers': [], # ok
                    'Screenplay': [], # ok
                    'State': [], 
                    'Language': [],
                    'Budget': [],
                    'Revenue': [],
                    'Keywords': [],
                    'Cast' : [] # ok
                    }
    return scraped_dict

In [231]:
#dict di prova!!!!!!!!!!!!!!!!!!!!!!!!
"""def empty_dict():
    scraped_dict = {  
                    'Title': [],
                    'Genre': [],
                    'Release_date': [],
                    'User_rating': [],
                    'Tagline': [],
                    'Cast': []
                    }
    return scraped_dict
"""

"def empty_dict():\n    scraped_dict = {  \n                    'Title': [],\n                    'Genre': [],\n                    'Release_date': [],\n                    'User_rating': [],\n                    'Tagline': [],\n                    'Cast': []\n                    }\n    return scraped_dict\n"

In [361]:
"""def user_score_info(tag_user_score, i, scraped_dict):
    if tag_user_score[i]['data-percent'] == '0':
        scraped_dict['User_rating'].append('Not rated yet')
    else:
        scraped_dict['User_rating'].append(tag_user_score[i]['data-percent'])"""

def user_score_info(tag_user_score, i):
    if not tag_user_score[i] or 'data-percent' not in tag_user_score[i]:
        return "Not rated yet"
    else:
        return tag_user_score[i]['data-percent']

In [233]:
doc.find_all('div', {'class': 'card style_1'})[0].h2.a['href']

'/movie/1096197-no-way-up'

In [234]:
def get_show_info(doc_page):
    base_link_1 = "https://www.themoviedb.org"
    tag_title = tag_premired_date = tag_shows_page = doc_page.find_all('div', {'class': 'card style_1'})
    tag_user_score = doc_page.find_all('div', {"user_score_chart"}) 
    
    doc_2_list = []
    for link in tag_shows_page:
        # here we are creating the list of all the individual pages of the shows which will come handy in other functions. 
        doc_2_list.append(get_page_content("https://www.themoviedb.org" + link.h2.a['href']))
       # we are going to have the function to return the list of all the information as elements. 
    return tag_title, tag_user_score, doc_2_list

In [235]:
#lets download and get the html of the individual page of the show 'what if...?' with the function get_page_content(). 
doc_2 = get_page_content("https://www.themoviedb.org/movie/10001")
tag_genre = doc_2.find('span', {"class": "genres"})
tag_genre_list = tag_genre.find_all('a')

check_genre =[]
for tag in tag_genre_list:
    check_genre.append(tag.text)

check_genre

['Commedia', 'Fantascienza']

In [236]:
# lets create a function to get the genres for the show. 
# i here denotes the element of the list vairable ``doc2_page`` that contains different doc pages. Will come handy later on.
def get_genres(doc2_page, i):
    genres_tags = doc2_page[i].find('span', {"class": "genres"}).find_all('a')
    check_genre =[]
    
    for tag in genres_tags:
        check_genre.append(tag.text)
    return check_genre

In [237]:
tag_tagline = doc_2.find('h3',{"class": 'tagline'})

def tagline_info(doc_2_list, i, scraped_dict):
    if doc_2_list[i].find('h3',{"class": 'tagline'}):
        scraped_dict['Tagline'].append(doc_2_list[i].find('h3',{"class": 'tagline'}).text)
    else:
        scraped_dict['Tagline'].append("No Tagline")

In [238]:
# i here denotes the the element of the list type variable``doc2_page`` that contains different doc pages.

def get_show_cast(doc2_page, i):
    cast_tags = doc2_page[i].find_all('li', {'class': 'card'})
    cast_lis = []
    
    for t in cast_tags:
         cast_lis.append(t.p.text)
    
    return cast_lis

In [317]:
def get_description(doc_2_list, i,scraped_dict): 
    if doc_2_list[i].find('div',{"class": 'overview'}):
        scraped_dict['Description'].append(doc_2_list[i].find('div',{"class": 'overview'}).p.text)
        # print(doc_2_list[i].find('div',{"class": 'overview'}).p.text)
    else:
        scraped_dict['Description'].append("No Description")

In [321]:
get_description(doc_2_list_,2,scraped_dict=empty_dict())

In [247]:
def convert_runtime_to_minutes(runtime):
    hours = 0
    minutes = 0
    
    # Dividi la stringa per spazi per separare le parti ore e minuti
    parts = runtime.split()
    
    for part in parts:
        if 'h' in part:
            # Estrai il numero di ore e convertilo in minuti
            hours = int(part.replace('h', '')) * 60
        elif 'm' in part:
            # Estrai il numero di minuti
            minutes = int(part.replace('m', ''))
    
    # Somma i minuti totali
    total_minutes = hours + minutes
    return total_minutes

def get_runtime(doc_2_list, i, scraped_dict): 
    if doc_2_list[i].find('span',{"class": 'runtime'}):
        scraped_dict['Runtime'].append(convert_runtime_to_minutes(doc_2_list[i].find('span',{"class":'runtime'}).text))
        #print(convert_runtime_to_minutes(doc_2_list[i].find('span',{"class":'runtime'}).text))
    else:
        scraped_dict['Runtime'].append("No Runtime")
        # print("No runtime")

In [242]:
def get_state(doc_2_list,i):
    # Cerca il tag `<strong>` che contiene il testo "Titolo originale"
    original_title_label = doc_2_list[i].find('section', class_='facts left_column')
    p_tags_list = []

    # Controlla se la sezione è stata trovata
    if original_title_label:
        # Trova tutti i tag <p> all'interno della sezione
        p_tags = original_title_label.find_all('p')
        # Aggiungi il testo di ogni tag <p> alla lista
        for p_tag in p_tags:
            p_tags_list.append(p_tag.get_text().strip()) 
        return parsing_state(p_tags_list)
    else:
        return  "Stato non trovato"

def parsing_state(p_tags_list):
    # Lista per i valori estratti
    extracted_values = ['None','None','None','None','None']
    
    # Ciclo per estrarre i valori
    for item in p_tags_list:
        # Dividi la stringa una volta e prendi il secondo elemento
        parts = item.split(' ', 1)
        if len(parts) > 1:
            type = parts[0]
            value = parts[1]
            if type == 'Stato':
                extracted_values[1] = value
            if type=='Lingua':
                value = value.replace('Originale','')
                extracted_values[2] = value
            if type=='Titolo':
                value = value.replace('originale','')
                extracted_values[0] = value
            if type=='Budget':
                value = value.replace('$', '').replace(',', '').strip()
                extracted_values[3] = value
            if type=='Incasso':
                value = value.replace('$', '').replace(',', '').strip()
                extracted_values[4] = value
    return extracted_values

In [333]:
def get_directors(doc_2_list,i,scraped_dict): 
    profiles = doc_2_list[i].find('ol',{"class": 'people no_image'}).find_all('li')
    for profile in profiles:
        name = profile.find('a').text  # Trova il tag <a> e ottieni il testo
        role = profile.find('p', class_='character').text  # Trova il <p> con classe "character" e ottieni il testo
        if role == 'Director':
            scraped_dict['Directors'].append(name)
        elif role == 'Screenplay':
            scraped_dict['Screenplay'].append(name)
        elif role == 'Writer':
            scraped_dict['Writers'].append(name)
        elif role == 'Novel':
            scraped_dict['Novel'].append(name)

In [335]:
def get_keywords(doc2_list,i, scraped_dict):
    keywords_section = doc2_list[i].find('section', class_='keywords right_column')
    keyword_lis = keywords_section.find_all('li')
    keywords = [li.get_text(strip=True) for li in keyword_lis]
    scraped_dict['keywords'] = keywords

In [336]:
def fillna(scraped_dict):
    print(scraped_dict)

fillna(empty_dict())

{'Title': [], 'Genre': [], 'Duration': [], 'Release_date': [], 'User_rating': [], 'Tagline': [], 'Description': [], 'Directors': [], 'Novel': [], 'Writers': [], 'Screenplay': [], 'State': [], 'Language': [], 'Budget': [], 'Revenue': [], 'Keywords': [], 'Cast': []}


In [349]:
def ensure_equal_length(scraped_dict):
    max_length = max(len(lst) for lst in scraped_dict.values())  # Trova la lunghezza massima delle liste
    for key, value_list in scraped_dict.items():
        current_length = len(value_list)
        if current_length < max_length:
            # Riempi la lista con 'None' o con un altro valore di default fino a raggiungere la lunghezza massima
            value_list.extend([None] * (max_length - current_length))

In [363]:
def get_show_details(t_title, t_user_score, docs_2_list):
    # excuting a function here that empties the dictionary every time the function is called.
    scraped_dict = {'Title': [], 'Genre': [], 'UserScore': [], 'Revenue': [], 'Cast': []}
    for i in range (0, len(t_title)):
        if t_title[i] is not None:
            title_text = t_title[i].h2.text if t_title[i].h2 else "Title Not Found"
            scraped_dict['Title'].append(title_text)
        else:
            scraped_dict['Title'].append("Title Not Found")
        # scraped_dict['Title'].append(t_title[i].h2.text)
        scraped_dict['Genre'].append(get_genres(docs_2_list, i))
        # user_score_info(t_user_score, i, scraped_dict)    
        scraped_dict['User_rating'].append(user_score_info(t_user_score, i))
        scraped_dict['Release_date'].append(t_title[i].p.text)
        tagline_info(docs_2_list, i, scraped_dict)  
        get_description(doc_2_list_,i,scraped_dict)
        get_directors(docs_2_list,i,scraped_dict)
        scraped_dict['State'].append(get_state(docs_2_list,i)[1])
        scraped_dict['Language'].append(get_state(docs_2_list,i)[2])
        scraped_dict['Budget'].append(get_state(docs_2_list,i)[3])
        scraped_dict['Revenue'].append(get_state(docs_2_list,i)[4])
        get_keywords(docs_2_list,i,scraped_dict)
        scraped_dict['Cast'].append(get_show_cast(docs_2_list, i))
        # Dopo aver popolato scraped_dict con i tuoi dati di scraping:
        ensure_equal_length(scraped_dict)
        df = pd.DataFrame(scraped_dict)
    return df

In [365]:
tag_title_, tag_user_score_, doc_2_list_ = get_show_info(doc)

In [366]:
# Let's excute our function to check if it works. We are going to take a look the data of dataframe.

x = get_show_details(tag_title_, tag_user_score_, doc_2_list_)
x.to_csv('newcheck.csv')
pd.read_csv('newcheck.csv',index_col=[0])

KeyError: 'User_rating'

In [ ]:
base_link = "https://www.themoviedb.org/movie"
import os
# 'i' here means the number of page we want to extract
def create_page_df( i, dataframe_list):
    os.makedirs('shows-data', exist_ok = True)
    next_url = base_link + '?page={}'.format(i)
    doc_top = get_page_content(next_url)
    name_tag, viewer_score_tag, doc_2_lis = get_show_info(doc_top)
    print('scraping page {} :- {}'.format(i, next_url))
    dataframe_data = get_show_details(name_tag, viewer_score_tag, doc_2_lis)
    dataframe_data.to_csv("shows-data/shows-page-{}.csv".format(i) , index = None)
    print(" ---> a CSV file with name shows-page-{}.csv has been created".format(i))
    dataframe_list.append(dataframe_data)

In [ ]:
test_list = []
create_page_df(9 , test_list)

In [ ]:
base_link = "https://www.themoviedb.org/movie"

def scrape_top_200_shows(base_link):
    dataframe_list = []
    # we are going to keep range up to 11 because we just need up to 200 TV shows for now. 
    for i in range(1,11):
        create_page_df(i, dataframe_list)
    # here we are using concat function so that we can merge the each dataframe that we got from the each page.    
    total_dataframe = pd.concat(dataframe_list, ignore_index = True)
    
    # with the simple command of to_csv() we can create a csv file of all the pages we extracted.
    csv_complete =  total_dataframe.to_csv('shows-data/Total-dataframe.csv', index= None)
    print(" \n a CSV file named Total-dataframe.csv with all the scraped shows has been created")

In [ ]:
scrape_top_200_shows(base_link)
pd.read_csv('shows-data/Total-dataframe.csv')[0:50]

NO SAFE ZONE